## Count Threatened Species Listed Per Year

In [ ]:
import pandas as pd

# Load the threatened fauna dataset
species_df = pd.read_excel("EPBC Act List of Threatened Fauna.xlsx", sheet_name="Sheet2")

# Extract the year from the 'Effective' date column
species_df['year'] = pd.to_datetime(species_df['Effective'], errors='coerce').dt.year

# Count the number of species listed per year
species_per_year = species_df.groupby('year').size().reset_index(name='species_count')

# Display the result
print(species_per_year.head(10))


FileNotFoundError: [Errno 2] No such file or directory: 'EPBC Act List of Threatened Fauna.xlsx'

## Add Urban Area Data

In [ ]:
# Load urban area data
urban_df = pd.read_csv("Urban_Area_Victoria_MODIS.csv")

# Keep only relevant columns
urban_df = urban_df[['year', 'urban_area_ha']]

# Merge with species_per_year
merged_df = species_per_year.merge(urban_df, on='year', how='left')

# Display result
print(merged_df.head(10))


## Add Burned Area Data

In [ ]:
# Load burned area data
burned_df = pd.read_csv("Victoria_Burned_Area_MODIS.csv")


# Group by year and sum the burned area
burned_df_cleaned = (
    burned_df.groupby('year')['burned_area_ha']
    .sum()
    .reset_index()
)

# Merge with the existing merged_df
merged_df = merged_df.merge(burned_df_cleaned, on='year', how='left')


# Display result
print(merged_df.head(10))


## Add Rainfall and Temperature

In [ ]:
# Load rainfall data
rainfall_df = pd.read_excel("rainfall_vicxlsx.xlsx")
rainfall_df = rainfall_df[['Year', 'Annual']].rename(columns={'Year': 'year', 'Annual': 'rainfall_annual_mm'})

# Load temperature data
temp_df = pd.read_excel("temp_vic.xlsx")
temp_df = temp_df[['Year', 'Annual']].rename(columns={'Year': 'year', 'Annual': 'temp_annual_c'})

# Merge both into the existing merged_df
merged_df = merged_df.merge(rainfall_df, on='year', how='left')
merged_df = merged_df.merge(temp_df, on='year', how='left')

# Show updated result
print(merged_df.head(10))


## Add Population Data

In [ ]:
# Load and skip metadata rows
pop_df = pd.read_excel("vic_pop.xlsx", sheet_name="Data1", skiprows=10)

# Rename the columns using visual confirmation
pop_df.columns = [
    'quarter',
    'pop_male_vic',
    'pop_male_aus',
    'pop_female_vic',
    'pop_female_aus',
    'pop_total_vic',
    'pop_total_aus'
]

# Drop rows with missing total Vic population
pop_df = pop_df[['quarter', 'pop_total_vic']].dropna()

# Extract year from the 'quarter' date
pop_df['year'] = pd.to_datetime(pop_df['quarter']).dt.year

# Average quarterly estimates per year
pop_annual = (
    pop_df.groupby('year')['pop_total_vic']
    .mean()
    .round()
    .astype(int)
    .reset_index()
    .rename(columns={'pop_total_vic': 'population'})
)

# Display result
print(pop_annual.head(10))

## Final Merge

In [ ]:
# Merge cleaned population data into the main dataset
merged_df = merged_df.merge(pop_annual, on='year', how='left')

# Show the final result
print(merged_df.head(15))

## Exporting to CSV

In [ ]:
merged_df.to_csv("prediction_dataset.csv", index=False)

In [ ]:
merged_df = pd.read_csv("prediction_dataset.csv")
